In [3]:
for ii in 1:4
    addprocs(["heathred"])
end

for ii in 1:3
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(2)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


18-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19

In [4]:
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [5]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE
@everywhere using Zlib

using ClusterSoup

In [1]:
using WordEmbeddings
LL,word_indexes, indexed_words = load_word2vec_embeddings("../../Resources/example_code/word2vec/GoogleNews-vectors-negative300.bin", 15000)
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(300,15000)
Dict{String,Int64}
Array{String,1}


In [6]:
training_trees = open("questionbank_sents.jsz","r") do fs
    deserialize(fs)
end;
@pz training_trees

training_trees		Array{Any,1}	(4000,)


In [7]:
r_training_trees = r_chunk_data(training_trees)

18-element Array{RemoteRef,1}:
 RemoteRef(2,1,338) 
 RemoteRef(3,1,340) 
 RemoteRef(4,1,342) 
 RemoteRef(5,1,344) 
 RemoteRef(6,1,346) 
 RemoteRef(7,1,348) 
 RemoteRef(8,1,350) 
 RemoteRef(9,1,352) 
 RemoteRef(10,1,354)
 RemoteRef(11,1,356)
 RemoteRef(12,1,358)
 RemoteRef(13,1,360)
 RemoteRef(14,1,362)
 RemoteRef(15,1,364)
 RemoteRef(16,1,366)
 RemoteRef(17,1,368)
 RemoteRef(18,1,370)
 RemoteRef(19,1,372)

In [8]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e[:] = θ[1: W_e_len]
    rae.b_e[:] = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d[:] = θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len]
    rae.b_d[:] = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::NumericMatrix, ∇b_e::NumericVector, ∇W_d::NumericMatrix, ∇b_d::NumericVector)
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end

In [9]:
function loss!(θ::Vector)  
    #warn("loss! not defined")
    grad = similar(θ)
    #loss_and_loss_grad!(θ::Vector)
    cached_loss_and_loss_grad!(θ, grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    cached_loss_and_loss_grad!(θ, grad)
end


rae_outer = RAE(LL,word_indexes,indexed_words,0.001);
@time r_rae_outers = put!(workers(), rae_outer, 2)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    @inbounds for r_rae_outer in r_rae_outers
        update_remote(r_rae_outer, rae->unpack!(rae, θ) )
    end
    
    function get_remote_loss_grad_function(r_rae_outer::RemoteRef)
        @assert r_rae_outer.where == myid()
        rae = fetch(r_rae_outer)        
        function loss_and_loss_grad(tree::(Any,Any))
            Δs, err = UnfoldingRAE.loss_and_loss_grad(rae, tree)
            [pack(Δs...), err]
        end
    end
    loss_and_loss_grads::Array{RemoteRef,1} = map(r_rae_outers) do r_raeouter
        remotecall(r_raeouter.where, get_remote_loss_grad_function, r_raeouter)::RemoteRef
    end
    
    
    ret = prechunked_mapreduce(r_training_trees, loss_and_loss_grads, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

elapsed time: 29.370976263 seconds (1098973156 bytes allocated, 5.31% gc time)


loss_and_loss_grad! (generic function with 1 method)

In [10]:
loss_and_loss_grad_cache = Dict{NumericVector,(Number, NumericVector)}()
loss_and_loss_grad_cache_hits = 0
loss_and_loss_grad_cache_misses = 0
function cached_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global loss_and_loss_grad_cache
    global loss_and_loss_grad_cache_hits
    global loss_and_loss_grad_cache_misses
    if haskey(loss_and_loss_grad_cache,θ)
        loss_and_loss_grad_cache_hits+=1
        err, grad[:] = loss_and_loss_grad_cache[θ]
        err
    else
        loss_and_loss_grad_cache_misses+=1
        err = loss_and_loss_grad!(θ, grad)
        loss_and_loss_grad_cache[θ] = (err, grad)
        err
    end
end

cached_loss_and_loss_grad! (generic function with 1 method)

In [ ]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = cached_loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_LBFGS, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 60)
min_objective!(opt, tracking_loss_and_loss_grad!)

#θ = pack(rae_outer)


(optf,optx,ret) = optimize!(opt,θ)


In [ ]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,cached_loss_and_loss_grad!)
θ = pack(rae_outer)
#θ=res.minimum
res = optimize(f, θ, method=:cg, show_trace = true, store_trace = true, iterations = 10);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged                       
@printval res.f_converged 
@printval res.trace
@printval loss_and_loss_grad_cache_hits
@printval loss_and_loss_grad_cache_misses

Iter     Function value   Gradient norm 


In [ ]:
open("rae1062.jsz","w") do fs
    serialize(fs, rae_outer)
end;


In [ ]:
unpack!(rae_outer, res.minimum)


In [ ]:
importall WordEmbeddings 
function show_best(embedder,ĉ::Embedding, nbest=20, similarity=WordEmbeddings.cosine_sim )
    candidates=WordEmbeddings.neighbour_sims(ĉ,embedder.L, similarity)  
    candidates[find(map(isnan, candidates))]= -Inf #If the zero vector was an option, it will be NaN similar so it is banned from winning
    best_cands = [ (findfirst(candidates,score), score)
                    for score in select(candidates,1:nbest, rev=true)[1:nbest]]
    vcat([[embedder.indexed_words[ii] round(score,2)] for (ii,score) in best_cands]...)
end

function show_bests(embedder,ĉs::Embeddings, nbest=20, similarity=WordEmbeddings.cosine_sim)
    hcat([show_best(embedder,ĉs[:,ii],nbest, similarity) for ii in 1:size(ĉs,2)]...)
end


In [ ]:
tr=(("What", ("are", "birds")), "?")
a = fold(rae_outer,tr)
b= unfold(rae_outer, a)
ĉs = hcat(map(leave-> leave.ĉ, b)...)
bests = show_bests(rae_outer, ĉs)

In [ ]:
haskey(rae_outer.word_index,"?")

In [ ]:
findfirst(candidates,NaN)

In [ ]:
select(candidates,1:nbest, rev=true)

In [ ]:
candidates